In [1]:
import pandas as pd; import numpy as np; import janitor as jn
pd.__version__

'2.2.0'

In [2]:
# events = pd.read_csv('/Users/samuel.oranyeli/Downloads/results.csv', parse_dates=['start','end']).iloc[:, 1:]

In [3]:
# a = (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     # ('id', 'id', '!='),
#     # ('audience','audience','>'),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

# a

In [4]:
C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2],
       keyy = range(1,8)
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15],
       keyy=range(8,0,-1)
       )
D=pd.DataFrame(D)

In [5]:
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=True)
)

([Index([3, 4, 6, 0, 5, 2, 1], dtype='int64'),
  Index([2, 3, 0, 5, 4, 6, 1], dtype='int64')],
 [array([0, 1, 1, 1, 1, 2, 3]), array([0, 0, 1, 1, 2, 2, 2])])

In [6]:
(C
.conditional_join(
    D, 
    
    ('profit','profit','>='),
    # ('keyy','keyy','=='),
  # ('unitsSold','unitsSold','>='),
  ('vol','vol','<='),
  # ('vol','profit','>'),
    use_numba=False)
)[0].size

19

In [7]:
from string import ascii_lowercase
np.random.seed(1)
n = 20; k = 20
n = 20_000_000; k = 20_000
mapp = {k:v for v,k in enumerate(ascii_lowercase)}
idx1 = np.random.randint(0, high = 2_00, size = n)
idx2 = np.random.randint(0, high = 3_00, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2),
          )


d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)
d1 = d1.assign(xx=lambda df: df.x.map(mapp))
d2 = d2.assign(xx=lambda df: df.x.map(mapp))

In [8]:
bla = (d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)

bla

([Index([19999568, 19999326, 19999177, 19998979, 19998939, 19997833, 19997813,
         19997525, 19997389, 19996211,
         ...
             1717,     1310,     1209,      982,      873,      726,      639,
              404,      282,      236],
        dtype='int64', length=17497550),
  Index([      26,      211,      654,     1296,     1342,     1468,     1557,
             1572,     2537,     3167,
         ...
         19998032, 19998224, 19998289, 19998808, 19999029, 19999066, 19999161,
         19999525, 19999797, 19999857],
        dtype='int64', length=18758834)],
 [array([ 0,  0,  0, ..., 90, 90, 90]), array([ 0,  0,  0, ..., 90, 90, 90])])

In [9]:
%%timeit
(d2
.conditional_join(
    d1, 
    ('pos1', 'start', '>'), 
     ('pos1', 'end', '<'), 
   # ('xx', 'xx', '=='), 
    use_numba=True)
)

6.05 s ± 83.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
# %timeit d1.sort_values('start')